# Avaliação – Produto Renda Fixa
## Construção de uma calculadora de renda fixa em Python - NTN-F


Formato de envio: arquivo .ipynb (Jupyter Notebook) conforme instruções a seguir.

- Pode construir quantos métodos/classes/funções adicionais desejarem;
- Pode utilizar bibliotecas abertas;
- O arquivo deve conter as funções solicitadas e os cenários de teste (pelo menos um teste para cada função/método). 
- Comente as funções e o que mais achar necessário no código;

Exemplo de definição da função com docstring detalhando os parâmetros:

```{python}
def calcula_soma(a, b):
''' Calcula a soma de dois números decimais
    a: float
    b: float
    '''
    return a + b
```

Exemplo de teste da função com a definição dos parâmetros e chamada da função:

```{python}
a = 1.3
b = 10.07
resultado = calcula_soma(a, b):
print(f'Resultado: {resultado}')
```

- A nota da avaliação considerará apenas o resultado das funções desejadas (existência de bugs, bom funcionamento, etc);
- Na avaliação qualitativa, faremos comentários de melhorias na estrutura do código (para fins de evolução de aprendizado, e não para reduzir nota).

### Funções/métodos mínimos de entrega:
- calcula_prazo(dt_ini, dt_fim, feriados, convencao)
    - Retorna: prazo anualizado (float)
    - convencao: 'DU/252' ou 'DC/360'
    
    
- constroi_fluxo(dt_fim, frequencia)
    - Retorna: Lista de datas dos fluxos (list datetime.date)
    
    
- calcula_pu(VF, prazo_anual, taxa_anual)
    - Retorna pu (float)
    
    
- calcula_taxa_anual(PU, prazo_anual, valor_base=100)
    - Retorna: taxa_anual (float)
    
    
- calcula_pu_ntnf(dt_venc, dt_base, tir) *
    - Retorna: pu (float)
    - Imprime tabela com o cashflow (Data do fluxo, VF, DU, Fator de desconto, PU)

\* semelhante ao calculado em aula no Excel. Sugere-se aproveitar as funções anteriores para este cálculo.

### Sugestões (itens não obrigatórios):
- Utilizar variáveis de data no formato datetime.date;
- Utilizar o calendário Anbima em .xls (disponibilizado no material de aula ou em: https://www.anbima.com.br/feriados/arqs/feriados_nacionais.xls) para cálculo de dias úteis;

### Itens adicionais:

Se você deseja incrementar seu framework para construção de portfólio pessoal, seguem algumas sugestões de melhorias para o projeto. 

Obs: Estes itens não serão considerados na nota, trabalhe neles apenas após ter garantido a parte obrigatória:

- Implementar diversas contagens de prazo. Ex: DU/252, 30/360, ACT/360, ACT/ACT. Ver padrões ISDA.
- Implementar tratamento de dias úteis/feriados. Por exemplo, se o vencimento de um fluxo cai num feriado/fim de semana, deslocar para o dia útil anterior ou próximo; escolher o critério via parâmetro;
- Calcular accrual. Ex: def calcula_accrual(dt_ini, dt_base, taxa_anual);
- Ler dados históricos (por exemplos, planilhas do tesouro direto) e fazer gráfico da série de preços de mercado ou das taxas;
- Fazer gráficos de séries históricas de preço de mercado e preço accruado no mesmo gráfico.
- Etc.


In [1]:
# Função datetime.timedelta

- calcula_prazo(dt_ini, dt_fim, feriados, convencao)
    - Retorna: prazo anualizado (float)
    - convencao: 'DU/252' ou 'DC/360'
    

## Criando função de cálculo de prazo

In [175]:
import pandas as pd
import numpy as np
from datetime import datetime

def calcula_prazo(dt_ini, dt_fim, feriados, convencao):
    f = '%d/%m/%Y'
    dif_uteis = np.busday_count(datetime.strptime(dt_ini, f).date(), datetime.strptime(dt_fim, f).date()) - feriados
    if convencao == 252:
        return dif_uteis/252
    elif convencao == 360:
        return dif/360
    else:
        return dif_uteis

In [155]:
s = '18/02/2022'
t = '01/01/2031'
f = '%d/%m/%Y'

In [156]:
# Aplicação da Função

calcula_prazo(s, t, 3, 252)

9.166666666666666

In [157]:
# Teste

dif = np.busday_count(datetime.strptime(s, f).date(), datetime.strptime(t, f).date())

(dif - 3) / 252

9.166666666666666

## Criando função para mostrar a lista de datas entre duas datas

In [93]:
from datetime import date, timedelta

def lista_dias(data_inicial, data_final):
    f = '%d/%m/%Y'
    d1 = datetime.strptime(data_inicial, f)
    d2 = datetime.strptime(data_final, f)

    # Padronizando as datas

    dias = pd.Series(pd.date_range(d1, d2-timedelta(days=1),freq='d').strftime('%Y-%m-%d'))

    return dias

In [7]:
dias = lista_dias('18/02/2022', '01/01/2031')
dias.head()

0    2022-02-18
1    2022-02-19
2    2022-02-20
3    2022-02-21
4    2022-02-22
dtype: object

In [8]:
dias.size

3239

## Importando a planilha de feriados nacionais e fazendo o tratamento

### Função Tratamento de Datas

In [9]:
import pandas as pd

df_fn = pd.read_excel('feriados_nacionais.xls')
df_fn.sample(10).head()

,Data,Dia da Semana,Feriado
7,2001-09-07 00:00:00,sexta-feira,Independência do Brasil
837,2070-11-02 00:00:00,domingo,Finados
563,2047-12-25 00:00:00,quarta-feira,Natal
347,2029-12-25 00:00:00,terça-feira,Natal
662,2056-02-15 00:00:00,terça-feira,Carnaval


In [10]:
df_fn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 945 entries, 0 to 944
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Data           941 non-null    object
 1   Dia da Semana  936 non-null    object
 2   Feriado        936 non-null    object
dtypes: object(3)
memory usage: 22.3+ KB


In [11]:
import pandas as pd

def clean_data(df):
    df = pd.to_datetime(df, errors='coerce')
    df = df.dropna(how = 'all')
    df = df.dt.strftime('%Y-%m-%d')

    return df

In [12]:
# Teste

lista_feriados = clean_data(df_fn['Data'])
lista_feriados.size

936

In [13]:
lista_feriados.head()

0    2001-01-01
1    2001-02-26
2    2001-02-27
3    2001-04-13
4    2001-04-21
Name: Data, dtype: object

### Função Lista com as Datas dos feriados entre o intervalo

In [14]:
def datas_feriados(data_dias, data_feriados):
    dias_feriados = pd.Series(data_dias.isin(data_feriados).values)
    dias_feriados[dias_feriados].index
    feriados = pd.Series(data_dias[dias_feriados].values)
    return feriados

In [15]:
# Teste

f_entre = datas_feriados(dias, lista_feriados)
f_entre

0      2022-02-28
1      2022-03-01
2      2022-04-15
3      2022-04-21
4      2022-05-01
          ...    
102    2030-09-07
103    2030-10-12
104    2030-11-02
105    2030-11-15
106    2030-12-25
Length: 107, dtype: object

In [16]:
# Número de Feriados

f_entre.count()

107

In [17]:
# Testando função calcula_prazo com função feriados

calcula_prazo('01/01/2022','01/01/2031', f_entre.count(), 252)

8.88888888888889

### Função para Lista de Dias sem os feriados

In [18]:
def sem_feriados(data_dias, data_feriados):
    lista_s_feriados = data_dias[~data_dias.isin(data_feriados)]
    return lista_s_feriados

In [19]:
# Teste

lista_sem_feriados = sem_feriados(dias, f_entre)
lista_sem_feriados

0       2022-02-18
1       2022-02-19
2       2022-02-20
3       2022-02-21
4       2022-02-22
           ...    
3234    2030-12-27
3235    2030-12-28
3236    2030-12-29
3237    2030-12-30
3238    2030-12-31
Length: 3132, dtype: object

In [20]:
# Número de dias sem feriados

lista_sem_feriados.count()

3132

In [21]:
# Conferindo o resultado

dias.count() - f_entre.count()

3132

- constroi_fluxo(dt_fim, frequencia)
    - Retorna: Lista de datas dos fluxos (list datetime.date)


## Função Lista de Datas dos Fluxos

### Pagamentos ocorrem em 01/01 e 01/07

In [22]:
def constroi_fluxo(dt_fm, frequencia):
    f = '%d/%m/%Y'
    d1 = date.today()
    d2 = datetime.strptime(dt_fm, f)
    dias_f = pd.Series(pd.date_range(d1, d2-timedelta(days=0), 
                                     freq=frequencia))
    return dias_f

In [23]:
# Teste

constroi_fluxo('01/01/2031', 'QS')

0    2022-04-01
1    2022-07-01
2    2022-10-01
3    2023-01-01
4    2023-04-01
5    2023-07-01
6    2023-10-01
7    2024-01-01
8    2024-04-01
9    2024-07-01
10   2024-10-01
11   2025-01-01
12   2025-04-01
13   2025-07-01
14   2025-10-01
15   2026-01-01
16   2026-04-01
17   2026-07-01
18   2026-10-01
19   2027-01-01
20   2027-04-01
21   2027-07-01
22   2027-10-01
23   2028-01-01
24   2028-04-01
25   2028-07-01
26   2028-10-01
27   2029-01-01
28   2029-04-01
29   2029-07-01
30   2029-10-01
31   2030-01-01
32   2030-04-01
33   2030-07-01
34   2030-10-01
35   2031-01-01
dtype: datetime64[ns]

In [24]:
constroi_fluxo('01/01/2031', 'QS')[1]

Timestamp('2022-07-01 00:00:00')

In [164]:
def data_semestre(s):
    for i in s:
        if i.month in [1,7]:
            pass
        else:
            s = s.drop(s[s == i].index[0])
    return s

In [165]:
s = constroi_fluxo('01/01/2031', 'QS')

In [166]:
data_semestre(s)

1    2022-07-01
3    2023-01-01
5    2023-07-01
7    2024-01-01
9    2024-07-01
11   2025-01-01
13   2025-07-01
15   2026-01-01
17   2026-07-01
19   2027-01-01
21   2027-07-01
23   2028-01-01
25   2028-07-01
27   2029-01-01
29   2029-07-01
31   2030-01-01
33   2030-07-01
35   2031-01-01
dtype: datetime64[ns]

In [167]:
data_semestre(s).count()

18

- calcula_pu(VF, prazo_anual, taxa_anual)
    - Retorna pu (float)


## Função Cálculo do PU

In [29]:
def calcula_pu(VF, prazo_anual, taxa_anual):
    PU = VF / (1 + taxa_anual)**prazo_anual
    return PU

In [30]:
calcula_pu(1000, 5.95/100, 15/252)

996.5656464615897

In [31]:
# Teste

VF = 1000
i = 5.95/100
n = 15/252

PU = VF / (1 + i)**n
PU

996.5656075679024

- calcula_taxa_anual(PU, prazo_anual, valor_base=100)
    - Retorna: taxa_anual (float)
    

## Taxa Anual

In [32]:
def calcula_taxa_anual(PU, prazo_anual, convencao):    
    VN = 1000
    a = (VN/PU)
    if convencao == 252:
        return (a ** (1/(prazo_anual*252)) - 1)/100
    elif convencao == 360:
        return (a ** (1/(prazo_anual*360)) - 1)/100

In [33]:
# Teste

calcula_taxa_anual(959, 5, 360)

2.325816163284422e-07

- calcula_pu_ntnf(dt_venc, dt_base, tir) *
    - Retorna: pu (float)
    - Imprime tabela com o cashflow (Data do fluxo, VF, DU, Fator de desconto, PU)

## Função que Cálcula o PU de NTN-F

In [34]:
# Função Cálculo do Cupom Semestral

def cupom_s(taxa):
    cupom = 1000*((1+(taxa/100))**(1/2)-1)
    return cupom

In [35]:
# Teste

cupom_s(10)

48.808848170151634

In [192]:
# Cálculo dos Dias Úteis entre Data_base e Data do Cupom

f = '%d/%m/%Y'
d1 = datetime.strptime('18/02/2022', f)

delta_s = []
for i in datas_cupom:
    delta_s.append(calcula_prazo('18/02/2022', i, 
                                 datas_feriados(lista_dias('18/02/2022', i), lista_feriados).count(), 0))
delta_s

[89,
 215,
 338,
 463,
 586,
 713,
 835,
 962,
 1084,
 1211,
 1333,
 1460,
 1583,
 1708,
 1831,
 1957,
 2079,
 2206]

In [193]:
# Trazendo os Cupons para Valor Presente

cupons_p = []

for i in delta_s:
    cupons_p.append(calcula_pu(cupom_s(10), i/252, 0.1148))
    
cupons_p

[46.9709983802708,
 44.486814736054846,
 42.18855978243947,
 39.97454317307897,
 37.90939887333251,
 35.888981068152894,
 34.04958369706919,
 32.234878447049894,
 30.58276270266278,
 28.95282500561559,
 27.46892246461997,
 26.00493987228039,
 24.661486036924078,
 23.367274051996617,
 22.16008595230639,
 20.98809206298208,
 19.912401413214106,
 18.851150864410425]

In [195]:
# Soma dos Cupons do Período Trazidos à Valor Presente

cupons_soma = sum(cupons_p)
cupons_soma

556.653698584461

In [188]:
247.3826468175846 + 556.653698584461

804.0363454020456

In [196]:
# Trazendo o VN para o Valor Presente

VN = 1000
data_ini = '18/02/2022'
data_venc = '01/01/2031'
feriados = f_entre.count()
tir = 0.1148

VN_PV = VN/((1+tir)**(calcula_prazo(data_ini, data_venc, feriados, 252)))
VN_PV

386.2240468919441

In [199]:
# Valor do PU da NTN-F

valor_obtido = VN_PV + cupons_soma

valor_obtido

942.8777454764052

In [200]:
# Diferença com o PU disponível no site do tesouro da ntnf_2031

ntnf_2031 = 937.85

ntnf_2031 - valor_obtido

-5.02774547640513